In [1]:
import tensorflow as tf
import numpy as np
import mlp.tf_utils as utils
from mlp.data_providers import AugmentedCIFAR10DataProvider, AugmentedCIFAR100DataProvider, CIFAR100DataProvider, CIFAR10DataProvider
from mlp.image_transforms import random_flip, random_crop, center_crop, random_flip_small
from mlp.Conv_models import ConvModel, TwoTaskConvModel
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

seed = 12345 
rng = np.random.RandomState(seed)

In [2]:
layer_dims = [3, 64, 128, 128, 128, _]
batch_size = 256

bifurcation_point = 3
joint_loss = False
max_pools = [True, True, False, True]
lrns = []
lrn_alphas = []
batch_norms = [True, True, True, True]
l1 = False
l2 = 5e-4
learning_rates = [2e-2]  
lrn_schedule = []
optimizer_params = [0.9, 0.999]  
activation = tf.nn.relu
dropout = False
epochs = 50
error = 'soft_max_cross_entropy'
image_size = 32  
optimiser = "GD"  
logs_dir = "tf-log"

In [3]:
mtl_models = []
b_models = []

for i, ratio in enumerate([1.]):
    tf.reset_default_graph()
    model_name = 'MTL_SGD_V1_with_' + str(ratio) + "_of_the_data"
    print("MODEL:- " + model_name)
    
    main_train_data = CIFAR100DataProvider(which_set='train',
                                                    batch_size=batch_size, shuffle_order=False)
    main_valid_data = CIFAR100DataProvider(which_set='valid', batch_size=batch_size, shuffle_order=False)

    aux_train_data = CIFAR10DataProvider(which_set='train',
                                                   batch_size=batch_size, shuffle_order=False)
    aux_valid_data = CIFAR10DataProvider(which_set='valid', batch_size=batch_size, shuffle_order=False)
    
    m_t_idx = utils.sample_data(main_train_data, ratio)
    m_v_idx = utils.sample_data(main_valid_data, ratio)
    _1 = utils.sample_data(aux_train_data, ratio, m_t_idx)
    _2 = utils.sample_data(aux_valid_data, ratio, m_v_idx)
    
    model = TwoTaskConvModel(conv_layer_dims=layer_dims, main_train_data=main_train_data, main_valid_data=main_valid_data,
                         aux_train_data=aux_train_data, aux_valid_data=aux_valid_data, L1=l1, L2=l2,
                         batch_size=batch_size, learning_rates=learning_rates, learning_rate_schedule=lrn_schedule,
                         optimiser=optimiser, max_pools=max_pools, lin_response_norms=lrns,
                         lin_response_alphas=lrn_alphas, batch_norms=batch_norms, bifurcation_point=bifurcation_point,
                         optimiser_params=optimizer_params, activation=activation, dropout=dropout,
                         epochs=epochs, error=error, input_image_size=image_size, name=model_name, joint_loss=joint_loss)
    
    model.create_network()
    model.initialize_network(logs_dir)
    model.train_model()

    mtl_models.append(model)
    
    tf.reset_default_graph()
    model_name = 'baseline_SGD_V1_with_' + str(ratio) + "_of_the_data"
    print("MODEL:- " + model_name)
    
    train_data = CIFAR10DataProvider(which_set='train', batch_size=batch_size)
    valid_data = CIFAR10DataProvider(which_set='valid', batch_size=batch_size)
    
    # update baseline data providers to have same samples 
    _3 = utils.sample_data(train_data, ratio, m_t_idx)
    _4 = utils.sample_data(valid_data, ratio, m_v_idx)
    
    model_b = ConvModel(conv_layer_dims=layer_dims, train_data=train_data, valid_data=valid_data,
                 batch_size=batch_size, learning_rates=learning_rates, learning_rate_schedule=[],
                 optimiser=optimiser, L1=l1, L2=l2,
                 max_pools=max_pools, lin_response_norms=lrns, lin_response_alphas=lrn_alphas, batch_norms=batch_norms,
                 optimiser_params=optimizer_params, activation=activation, dropout=dropout,
                 epochs=epochs, error=error, input_image_size=image_size, name=model_name)
    model_b.create_network()
    model_b.initialize_network(logs_dir)
    model_b.train_model()
    
    b_models.append(model_b)

MODEL:- MTL_SGD_V1_with_1.0_of_the_data
Setting up model...
Initializing network...
Training model...
Epoch finished:  1
Epoch finished:  2
Epoch finished:  3
Epoch finished:  4
Epoch finished:  5
Epoch finished:  6
Epoch finished:  7
Epoch finished:  8
Epoch finished:  9
Epoch finished:  10
Epoch finished:  11
Epoch finished:  12
Epoch finished:  13
Epoch finished:  14
Epoch finished:  15
Epoch finished:  16
Epoch finished:  17
Epoch finished:  18
Epoch finished:  19
Epoch finished:  20
Epoch finished:  21
Epoch finished:  22
Epoch finished:  23
Epoch finished:  24
Epoch finished:  25
Epoch finished:  26
Epoch finished:  27
Epoch finished:  28
Epoch finished:  29
Epoch finished:  30
Epoch finished:  31
Epoch finished:  32
Epoch finished:  33
Epoch finished:  34
Epoch finished:  35
Epoch finished:  36
Epoch finished:  37
Epoch finished:  38
Epoch finished:  39
Epoch finished:  40
Epoch finished:  41
Epoch finished:  42
Epoch finished:  43
Epoch finished:  44
Epoch finished:  45
Epoch f

In [4]:
b = b_models[0]
for v in b.train_variables:
    print(v.op.name)
print( "  -----------   ")
m = mtl_models[0]
for v in m.train_variables:
    print(v.op.name)

baseline_SGD_V1_with_1.0_of_the_data_conv_model/W_0
baseline_SGD_V1_with_1.0_of_the_data_conv_model/b_0
baseline_SGD_V1_with_1.0_of_the_data_conv_model/BatchNorm/beta
baseline_SGD_V1_with_1.0_of_the_data_conv_model/W_1
baseline_SGD_V1_with_1.0_of_the_data_conv_model/b_1
baseline_SGD_V1_with_1.0_of_the_data_conv_model/BatchNorm_1/beta
baseline_SGD_V1_with_1.0_of_the_data_conv_model/W_2
baseline_SGD_V1_with_1.0_of_the_data_conv_model/b_2
baseline_SGD_V1_with_1.0_of_the_data_conv_model/BatchNorm_2/beta
baseline_SGD_V1_with_1.0_of_the_data_conv_model/W_3
baseline_SGD_V1_with_1.0_of_the_data_conv_model/b_3
baseline_SGD_V1_with_1.0_of_the_data_conv_model/BatchNorm_3/beta
baseline_SGD_V1_with_1.0_of_the_data_conv_model/W_pred
baseline_SGD_V1_with_1.0_of_the_data_conv_model/b_pred
  -----------   
MTL_SGD_V1_with_1.0_of_the_data_conv_model/W_0_shared
MTL_SGD_V1_with_1.0_of_the_data_conv_model/b_0_shared
MTL_SGD_V1_with_1.0_of_the_data_conv_model/BatchNorm/beta
MTL_SGD_V1_with_1.0_of_the_data_c